In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm , trange
%matplotlib inline
import math
from qiskit import QuantumCircuit,execute,Aer,IBMQ,QuantumRegister,ClassicalRegister
from openpyxl import Workbook
from openpyxl import load_workbook
provider = IBMQ.load_account()
import ad_diff_module as adm

In [2]:
# 使う量子ビット数を定義
qnum_dic = {
    'qnum' : 14,
    'qnum_after_decimal' : 4,
    'qnum_const' : 4,
    'qnum_interger' : 10
}

# 定数設定
const_dic = {
    'dt' : 1, 
    'dx' : 1,
    'x' : 40,
    't' : 10,
    'C' : 0.125,
    'D' : 0.25
}
C = const_dic['C']
D = const_dic['D']

qnum = qnum_dic['qnum']
qnum_after_decimal = qnum_dic['qnum_after_decimal']
qnum_const = qnum_dic['qnum_const']
qnum_interger = qnum_dic['qnum_interger']

# 量子回路実行設定
execute_setting = {
    'provider' : provider,
    'shots' : 1,
    'simulator' : 'simulator_mps'
}

# 回路作成用インデックス
index_dic = {
    'x':[i for i in range(qnum)],
    'x_c':[i for i in range(qnum,qnum+qnum_const)],
    'y':[i for i in range(qnum+qnum_const,qnum+qnum_const+qnum)],
    'y_c':[i for i in range(qnum+qnum_const+qnum,qnum+qnum_const+qnum+qnum_const)],
    'z':[i for i in range(qnum+qnum_const+qnum+qnum_const,qnum+qnum_const+qnum+qnum_const+qnum)],
    'z_c':[i for i in range(qnum+qnum_const+qnum+qnum_const+qnum,qnum+qnum_const+qnum+qnum_const+qnum+qnum_const)],
    'ans':[i for i in range(qnum+qnum_const+qnum+qnum_const+qnum+qnum_const,qnum+qnum_const+qnum+qnum_const+qnum+qnum_const+qnum)]
}

In [3]:
# エクセル設定
ABC = []
for i in range(26):
    ABC.append(chr(ord('A')+i))
for j in range(4):
    for i in range(26):
        ABC.append(ABC[j]+ABC[i])
while len(ABC)>const_dic['x']:
    ABC.pop()
excel_columns = ABC
excel_index = [str(i+1) for i in range(500)]
# 使用するエクセルブックのパスをここで設定。
file_path = './result_data/ad_diff_out.xlsx'
# 使用するエクセルブックのシートネームを指定
sheet_name = '量子'

In [4]:
%%time
ad_diff_circ = adm.ad_diff_circuit(qnum_dic,const_dic,index_dic)

for i in trange(40):
    wb = load_workbook(filename=file_path)
    ws = wb[sheet_name]
    data_list = []
    for j in excel_columns:
        data_list.append(ws[j+excel_index[i]].value)
    # 数合わせ
    data_list.append(0)
    data_list.insert(0,0)

    circ_list = []
    for k in range(const_dic['x']):
        length = qnum*4+qnum_const*3
        qc = QuantumCircuit(length,qnum)
        adm.encoding_three(qc,data_list[k],data_list[k+1],data_list[k+2],index_dic,qnum_after_decimal,qnum_interger)
        qc.append(ad_diff_circ,[_ for _ in range(length)])
        qc.measure(index_dic['ans'],[_ for _ in range(qnum)])
        circ_list.append(qc)

    # #リストで1ｓtep解が得られる。   
    quantum_answer = adm.execute_qc(circ_list, provider, 1, 'simulator_mps')
    new_data_list = [adm.ans_to_syousuu_sitei(adm.quant_ans_to_bin_ans(i),qnum_after_decimal*2) for i in quantum_answer]

    for j,column_num in enumerate(excel_columns):
        ws[column_num+excel_index[i+1]] = new_data_list[j]
    wb.save(file_path)

100%|████████████████████████████████████████| 40/40 [1:35:20<00:00, 143.01s/it]

CPU times: user 1h 56s, sys: 18.3 s, total: 1h 1min 14s
Wall time: 1h 35min 21s


In [12]:
# data_list = []
data_list

[1, 2, 3]

In [11]:
data_list.append(3)

In [17]:
type('1')

str

In [18]:
'1' + 1

TypeError: can only concatenate str (not "int") to str

In [6]:
# %%time
# ################################### メインプログラム ##########################################

# for i in trange(3):
#     # エクセルから1stepデータを読み込みリストにしまう作業
#     wb = load_workbook(filename=file_path)
#     ws = wb['Sheet1']
#     data_list = []
#     for j in excel_columns:
#         data_list.append(ws[j+excel_index[i]].value)

#     # 次ステップの拡散計算ジョブ投入
#     new_data_list = addif.solve_ad_diff_1step(data_list,qnum,qnum_const,qnum_after_decimal,qnum_interger,index_dic,C,D,x,provider,3,'simulator_mps')
    
#     # データリストをエクセルに書き込み保存する作業
#     for j,column_num in enumerate(excel_columns):
#         ws[column_num+excel_index[i+1]] = new_data_list[j]
#     wb.save(file_path)

In [2]:
ans = {0:10,1:990}

In [3]:
for a in ans:
    print(a)

0
1


In [ ]:
1
